In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import os
import pickle
from sklearn.metrics import confusion_matrix
import requests, bs4
from sklearn.metrics import roc_curve, auc

In [48]:
db = pd.read_csv('2017_2019stats.csv')

In [49]:
db19 = pd.read_csv('2019stats.csv')

### Training gradient boosting classifier on 2017-2019 data ###
Model Variations: 3 game rolling averages, 9 game rolling averages, dogs, home dogs, road dogs

In [50]:
feature3Cols = ['Location','Rolling3EFG','Rolling3TOV','Rolling3ORB','Rolling3FTR','Rolling3Pace','Opp R3Pace','Opp R3EFG','Opp R3TOV','Opp R3ORB','Opp R3FTR']
feature9Cols = ['Location','Rolling9EFG','Rolling9TOV','Rolling9ORB','Rolling9FTR','Rolling9Pace','Opp R9Pace','Opp R9EFG','Opp R9TOV','Opp R9ORB','Opp R9FTR']

target = ['Result']
X3 = db.dropna()[feature3Cols]
X9 = db.dropna()[feature9Cols]
y = np.array(db.dropna()[target]).flatten()

dogs = db[db['Line'] > 0].dropna()[feature3Cols]
Ydogs = np.array(db[db['Line'] > 0].dropna()['Result']).flatten()
homeDogs = db[(db['Line'] > 0) & (db['Location'] == 1)].dropna()[feature3Cols]
YhomeDogs = np.array(db[(db['Line'] > 0) & (db['Location'] == 1)].dropna()['Result']).flatten()
roadDogs = db[(db['Line'] > 0) & (db['Location'] == 0)].dropna()[feature3Cols]
YroadDogs = np.array(db[(db['Line'] > 0) & (db['Location'] == 0)].dropna()['Result']).flatten()

x9_train, x9_test, y9_train, y9_test = train_test_split(X9, y, test_size=0.3, random_state=3)
x3_train, x3_test, y3_train, y3_test = train_test_split(X3, y, test_size=0.3, random_state=3)
dog_train, dog_test, Ydog_train, Ydog_test = train_test_split(dogs, Ydogs, test_size = 0.3, random_state = 3)
homeDog_train, homeDog_test, YhomeDog_train, YhomeDog_test = train_test_split(homeDogs, YhomeDogs, test_size = 0.3, random_state = 3)
roadDog_train, roadDog_test, YroadDog_train, YroadDog_test = train_test_split(roadDogs, YroadDogs, test_size = 0.3, random_state = 3)

In [51]:
clfgtb9 = GradientBoostingClassifier(n_estimators=1500, learning_rate=0.1, max_depth=1, random_state=0).fit(x9_train, y9_train)
pred = clfgtb9.predict(x9_test)
print('All picks (R9):',clfgtb9.score(x9_test, y9_test))
print('All picks (R9) matrix:\n',confusion_matrix(y9_test, pred))
print(' ')

clfgtb3 = GradientBoostingClassifier(n_estimators=1500, learning_rate=0.1, max_depth=1, random_state=0).fit(x3_train, y3_train)
pred = clfgtb3.predict(x3_test)
print('All picks (R3):',clfgtb3.score(x3_test, y3_test))
print('All picks (R3) matrix:\n',confusion_matrix(y3_test, pred))
print(' ')

clfgtb_dog = GradientBoostingClassifier(n_estimators=1500, learning_rate=0.1, max_depth=1, random_state=0).fit(dog_train, Ydog_train)
pred = clfgtb_dog.predict(dog_test)
print('Dog picks:',clfgtb_dog.score(dog_test, Ydog_test))
print('Dog picks matrix:\n',confusion_matrix(Ydog_test, pred))
print(' ')

clfgtb_homedog = GradientBoostingClassifier(n_estimators=1500, learning_rate=0.1, max_depth=1, random_state=0).fit(homeDog_train, YhomeDog_train)
pred = clfgtb_homedog.predict(homeDog_test)
print('Home Dog picks:',clfgtb_homedog.score(homeDog_test, YhomeDog_test))
print('Home Dog picks matrix:\n',confusion_matrix(YhomeDog_test, pred))
print(' ')

clfgtb_roaddog = GradientBoostingClassifier(n_estimators=1500, learning_rate=0.1, max_depth=1, random_state=0).fit(roadDog_train, YroadDog_train)
pred = clfgtb_roaddog.predict(roadDog_test)
print('Road Dog picks:',clfgtb_roaddog.score(roadDog_test, YroadDog_test))
print('Road Dog picks matrix:\n',confusion_matrix(YroadDog_test, pred))

All picks (R9): 0.6073059360730594
All picks (R9) matrix:
 [[207 143]
 [115 192]]
 
All picks (R3): 0.5920852359208524
All picks (R3) matrix:
 [[199 151]
 [117 190]]
 
Dog picks: 0.6459627329192547
Dog picks matrix:
 [[184  32]
 [ 82  24]]
 
Home Dog picks: 0.5045871559633027
Home Dog picks matrix:
 [[44 22]
 [32 11]]
 
Road Dog picks: 0.677570093457944
Road Dog picks matrix:
 [[131  18]
 [ 51  14]]


In [52]:
dogs2019 = db19[db19['Line'] > 0].dropna().reset_index()
Ydogs2019 = db19[db19['Line'] > 0].dropna()['Result']

In [53]:
print("Using dog data[rolling 9 averages]: \n")

print("Using 9 game standard predictor: ")
pred = clfgtb9.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb9.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using 3 game standard predictor: ")
pred = clfgtb3.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb3.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using dog predictor: ")
pred = clfgtb_dog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb_dog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using home dog predictor: ")
pred = clfgtb_homedog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb_homedog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using road dog predictor: ")
pred = clfgtb_roaddog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb_roaddog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

Using dog data[rolling 9 averages]: 

Using 9 game standard predictor: 
Dog picks: 0.6980428704566636
Dog picks matrix:
 [[557 159]
 [165 192]]

Using 3 game standard predictor: 
Dog picks: 0.6150978564771669
Dog picks matrix:
 [[513 203]
 [210 147]]

Using dog predictor: 
Dog picks: 0.6495806150978565
Dog picks matrix:
 [[647  69]
 [307  50]]

Using home dog predictor: 
Dog picks: 0.6113699906803355
Dog picks matrix:
 [[541 175]
 [242 115]]

Using road dog predictor: 
Dog picks: 0.6421248835041938
Dog picks matrix:
 [[629  87]
 [297  60]]



The 9 game standard predictor picking winning dogs is 192-159; pretty good

In [57]:
print("Using rolling 3 averages: \n")

print("Using 9 game standard predictor: ")
pred = clfgtb9.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb9.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using 3 game standard predictor: ")
pred = clfgtb3.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb3.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using dog predictor: ")
pred1 = clfgtb_dog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb_dog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred1))
print('')

print("Using home dog predictor: ")
pred = clfgtb_homedog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb_homedog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using road dog predictor: ")
pred = clfgtb_roaddog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb_roaddog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

Using rolling 3 averages: 

Using 9 game standard predictor: 
Dog picks: 0.5927306616961789
Dog picks matrix:
 [[502 214]
 [223 134]]

Using 3 game standard predictor: 
Dog picks: 0.6728797763280522
Dog picks matrix:
 [[529 187]
 [164 193]]

Using dog predictor: 
Dog picks: 0.7465051258154707
Dog picks matrix:
 [[663  53]
 [219 138]]

Using home dog predictor: 
Dog picks: 0.6635601118359739
Dog picks matrix:
 [[539 177]
 [184 173]]

Using road dog predictor: 
Dog picks: 0.7269338303821062
Dog picks matrix:
 [[656  60]
 [233 124]]



The dog predictor goes 138-53 using dog data. That's fucking dope.

This data may not be valid - 2019 data is incorporated into the model. Need to use 2016-2018 data to predict 2019 games, and see how it holds up.

Also need to simulate the 2019 season playing out and scraping those new games/incorporating new data in real time

In [55]:
odds = []
for i, n in enumerate(pred1):
    if n == 1:
        odds.append(dogs2019['Odds'][i])

In [56]:
import statistics
statistics.mean(odds)

193.8848167539267

Average odds for all dog predictions was +193.

Betting 10 on each 'yes' prediction is 1,910 spent on total bets, with 23 returned on each win.

Winnings: 3174    Losses: 530    Profits: 2644

### Check results using 2017-2018 data on 2019 games 

In [197]:
db = pd.read_csv('2017_2018stats.csv')

In [198]:
feature3Cols = ['Location','Rolling3EFG','Rolling3TOV','Rolling3ORB','Rolling3FTR','Rolling3Pace','Opp R3Pace','Opp R3EFG','Opp R3TOV','Opp R3ORB','Opp R3FTR']
feature9Cols = ['Location','Rolling9EFG','Rolling9TOV','Rolling9ORB','Rolling9FTR','Rolling9Pace','Opp R9Pace','Opp R9EFG','Opp R9TOV','Opp R9ORB','Opp R9FTR']

target = ['Result']
X3 = db.dropna()[feature3Cols]
X9 = db.dropna()[feature9Cols]
y = np.array(db.dropna()[target]).flatten()

dogs3 = db[db['Line'] > 0].dropna()[feature3Cols]
Ydogs3 = np.array(db[db['Line'] > 0].dropna()['Result']).flatten()
homeDogs3 = db[(db['Line'] > 0) & (db['Location'] == 1)].dropna()[feature3Cols]
YhomeDogs3 = np.array(db[(db['Line'] > 0) & (db['Location'] == 1)].dropna()['Result']).flatten()
roadDogs3 = db[(db['Line'] > 0) & (db['Location'] == 0)].dropna()[feature3Cols]
YroadDogs3 = np.array(db[(db['Line'] > 0) & (db['Location'] == 0)].dropna()['Result']).flatten()

dogs9 = db[db['Line'] > 0].dropna()[feature9Cols]
Ydogs9 = np.array(db[db['Line'] > 0].dropna()['Result']).flatten()
homeDogs9 = db[(db['Line'] > 0) & (db['Location'] == 1)].dropna()[feature9Cols]
YhomeDogs9 = np.array(db[(db['Line'] > 0) & (db['Location'] == 1)].dropna()['Result']).flatten()
roadDogs9 = db[(db['Line'] > 0) & (db['Location'] == 0)].dropna()[feature9Cols]
YroadDogs9 = np.array(db[(db['Line'] > 0) & (db['Location'] == 0)].dropna()['Result']).flatten()

x9_train, x9_test, y9_train, y9_test = train_test_split(X9, y, test_size=0.3, random_state=1)
x3_train, x3_test, y3_train, y3_test = train_test_split(X3, y, test_size=0.3, random_state=2)

dog3_train, dog3_test, Ydog3_train, Ydog3_test = train_test_split(dogs3, Ydogs3, test_size = 0.3, random_state = 1)
homeDog3_train, homeDog3_test, YhomeDog3_train, YhomeDog3_test = train_test_split(homeDogs3, YhomeDogs3, test_size = 0.3, random_state = 1)
roadDog3_train, roadDog3_test, YroadDog3_train, YroadDog3_test = train_test_split(roadDogs3, YroadDogs3, test_size = 0.3, random_state = 1)

dog9_train, dog9_test, Ydog9_train, Ydog9_test = train_test_split(dogs9, Ydogs9, test_size = 0.3, random_state = 6)
homeDog9_train, homeDog9_test, YhomeDog9_train, YhomeDog9_test = train_test_split(homeDogs9, YhomeDogs9, test_size = 0.3, random_state = 6)
roadDog9_train, roadDog9_test, YroadDog9_train, YroadDog9_test = train_test_split(roadDogs9, YroadDogs9, test_size = 0.3, random_state = 6)

### Training models

In [199]:
clfgtb9 = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(x9_train, y9_train)
pred = clfgtb9.predict(x9_test)
print('All picks (R9):',clfgtb9.score(x9_test, y9_test))
print('All picks (R9) matrix:\n',confusion_matrix(y9_test, pred))
print(' ')
false_positive_rate, true_positive_rate, thresholds = roc_curve(y9_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)

clfgtb3 = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(x3_train, y3_train)
pred = clfgtb3.predict(x3_test)
print('All picks (R3):',clfgtb3.score(x3_test, y3_test))
print('All picks (R3) matrix:\n',confusion_matrix(y3_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(y3_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb3_dog = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(dog3_train, Ydog3_train)
pred = clfgtb3_dog.predict(dog3_test)
print('Dog (R3) picks:',clfgtb3_dog.score(dog3_test, Ydog3_test))
print('Dog (R3) picks matrix:\n',confusion_matrix(Ydog3_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ydog3_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb3_homedog = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(homeDog3_train, YhomeDog3_train)
pred = clfgtb3_homedog.predict(homeDog3_test)
print('Home Dog (R3) picks:',clfgtb3_homedog.score(homeDog3_test, YhomeDog3_test))
print('Home Dog (R3) picks matrix:\n',confusion_matrix(YhomeDog3_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(YhomeDog3_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb3_roaddog = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(roadDog3_train, YroadDog3_train)
pred = clfgtb3_roaddog.predict(roadDog3_test)
print('Road Dog (R3) picks:',clfgtb3_roaddog.score(roadDog3_test, YroadDog3_test))
print('Road Dog (R3) picks matrix:\n',confusion_matrix(YroadDog3_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(YroadDog3_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb9_dog = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(dog9_train, Ydog9_train)
pred = clfgtb9_dog.predict(dog9_test)
print('Dog (R9) picks:',clfgtb9_dog.score(dog9_test, Ydog9_test))
print('Dog (R9) picks matrix:\n',confusion_matrix(Ydog9_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ydog9_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb9_homedog = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(homeDog9_train, YhomeDog9_train)
pred = clfgtb9_homedog.predict(homeDog9_test)
print('Home (R9) Dog picks:',clfgtb9_homedog.score(homeDog9_test, YhomeDog9_test))
print('Home (R9) Dog picks matrix:\n',confusion_matrix(YhomeDog9_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(YhomeDog9_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb9_roaddog = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(roadDog9_train, YroadDog9_train)
pred = clfgtb9_roaddog.predict(roadDog9_test)
print('Road Dog (R9) picks:',clfgtb9_roaddog.score(roadDog9_test, YroadDog9_test))
print('Road Dog (R9) picks matrix:\n',confusion_matrix(YroadDog9_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(YroadDog9_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)

All picks (R9): 0.6133942161339422
All picks (R9) matrix:
 [[404 246]
 [262 402]]
 
0.6134800741427248
All picks (R3): 0.6164383561643836
All picks (R3) matrix:
 [[402 247]
 [257 408]]
0.6164741592038647
 
Dog (R3) picks: 0.6542635658914728
Dog (R3) picks matrix:
 [[404  25]
 [198  18]]
0.5125291375291375
 
Home Dog (R3) picks: 0.6590909090909091
Home Dog (R3) picks matrix:
 [[120  17]
 [ 58  25]]
0.5885586140181164
 
Road Dog (R3) picks: 0.6572769953051644
Road Dog (R3) picks matrix:
 [[271  18]
 [128   9]]
0.5017048468163564
 
Dog (R9) picks: 0.6775193798449612
Dog (R9) picks matrix:
 [[416  21]
 [187  21]]
0.5264533092765359
 
Home (R9) Dog picks: 0.6
Home (R9) Dog picks matrix:
 [[108  26]
 [ 62  24]]
0.542519958347796
 
Road Dog (R9) picks: 0.6643192488262911
Road Dog (R9) picks matrix:
 [[268  16]
 [127  15]]
0.5246478873239437


### 2019 test data

In [200]:
dogs2019 = db19[(db19['Odds'] > 110)].dropna().reset_index()
Ydogs2019 = db19[(db19['Odds'] > 110)].dropna()['Result']

In [202]:
print("Using dog data[rolling 9 averages]: \n")

print("Using 9 game standard predictor: ")
pred1 = clfgtb9.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb9.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred1))
print('')

print("Using 3 game standard predictor: ")
pred = clfgtb3.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb3.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R3 dog predictor: ")
pred = clfgtb3_dog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb3_dog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R3 home dog predictor: ")
pred = clfgtb3_homedog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb3_homedog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R3 road dog predictor: ")
pred = clfgtb3_roaddog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb3_roaddog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 dog predictor: ")
pred = clfgtb9_dog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb9_dog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 home dog predictor: ")
pred = clfgtb9_homedog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb9_homedog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 road dog predictor: ")
pred = clfgtb9_roaddog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb9_roaddog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

Using dog data[rolling 9 averages]: 

Using 9 game standard predictor: 
Dog picks: 0.6406593406593407
Dog picks matrix:
 [[491 143]
 [184  92]]

Using 3 game standard predictor: 
Dog picks: 0.6307692307692307
Dog picks matrix:
 [[499 135]
 [201  75]]

Using R3 dog predictor: 
Dog picks: 0.6912087912087912
Dog picks matrix:
 [[626   8]
 [273   3]]

Using R3 home dog predictor: 
Dog picks: 0.6802197802197802
Dog picks matrix:
 [[599  35]
 [256  20]]

Using R3 road dog predictor: 
Dog picks: 0.6879120879120879
Dog picks matrix:
 [[623  11]
 [273   3]]

Using R9 dog predictor: 
Dog picks: 0.6824175824175824
Dog picks matrix:
 [[607  27]
 [262  14]]

Using R9 home dog predictor: 
Dog picks: 0.6692307692307692
Dog picks matrix:
 [[585  49]
 [252  24]]

Using R9 road dog predictor: 
Dog picks: 0.6912087912087912
Dog picks matrix:
 [[619  15]
 [266  10]]



In [204]:
odds = []
for i, n in enumerate(pred1):
    if n == 1:
        odds.append(dogs2019['Odds'][i])
        
statistics.mean(odds)

228.17872340425532

Nothing really great here.


In [205]:
print("Using rolling 3 averages: \n")

print("Using 9 game standard predictor: ")
pred1 = clfgtb9.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb9.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred1))
print('')

print("Using 3 game standard predictor: ")
pred2 = clfgtb3.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb3.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred2))
print('')

print("Using R3 dog predictor: ")
pred = clfgtb3_dog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb3_dog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R3 home dog predictor: ")
pred = clfgtb3_homedog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb3_homedog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R3 road dog predictor: ")
pred = clfgtb3_roaddog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb3_roaddog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 dog predictor: ")
pred = clfgtb9_dog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb9_dog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 home dog predictor: ")
pred = clfgtb9_homedog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb9_homedog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 road dog predictor: ")
pred = clfgtb9_roaddog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb9_roaddog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

Using rolling 3 averages: 

Using 9 game standard predictor: 
Dog picks: 0.610989010989011
Dog picks matrix:
 [[436 198]
 [156 120]]

Using 3 game standard predictor: 
Dog picks: 0.6164835164835165
Dog picks matrix:
 [[452 182]
 [167 109]]

Using R3 dog predictor: 
Dog picks: 0.6692307692307692
Dog picks matrix:
 [[590  44]
 [257  19]]

Using R3 home dog predictor: 
Dog picks: 0.643956043956044
Dog picks matrix:
 [[535  99]
 [225  51]]

Using R3 road dog predictor: 
Dog picks: 0.667032967032967
Dog picks matrix:
 [[588  46]
 [257  19]]

Using R9 dog predictor: 
Dog picks: 0.6461538461538462
Dog picks matrix:
 [[522 112]
 [210  66]]

Using R9 home dog predictor: 
Dog picks: 0.6351648351648351
Dog picks matrix:
 [[507 127]
 [205  71]]

Using R9 road dog predictor: 
Dog picks: 0.6637362637362637
Dog picks matrix:
 [[566  68]
 [238  38]]



In [206]:
odds = []
for i, n in enumerate(pred1):
    if n == 1:
        odds.append(dogs2019['Odds'][i])
        
print(statistics.mean(odds))
print(statistics.median(odds))

odds = []
for i, n in enumerate(pred2):
    if n == 1:
        odds.append(dogs2019['Odds'][i])
        
print(statistics.mean(odds))
print(statistics.median(odds))

246.69182389937106
180.0
247.8831615120275
185.0


#### Predictor 2 above goes 109-182 with an average odds of 247, that's pretty good and gives a 30% ROI on flat bets

##### Now what I wanna do is build a model off 2018 data while continuously rolling in 2019 data as if the season was ongoing

In [207]:
db = pd.read_csv('2018stats.csv')

In [217]:
feature3Cols = ['Location','Rolling3EFG','Rolling3TOV','Rolling3ORB','Rolling3FTR','Rolling3Pace','Opp R3Pace','Opp R3EFG','Opp R3TOV','Opp R3ORB','Opp R3FTR']
feature9Cols = ['Location','Rolling9EFG','Rolling9TOV','Rolling9ORB','Rolling9FTR','Rolling9Pace','Opp R9Pace','Opp R9EFG','Opp R9TOV','Opp R9ORB','Opp R9FTR']

target = ['Result']
X3 = db.dropna()[feature3Cols]
X9 = db.dropna()[feature9Cols]
y = np.array(db.dropna()[target]).flatten()

dogs = db[db['Odds'] > 110].dropna()[feature3Cols]
Ydogs = np.array(db[db['Odds'] > 110].dropna()['Result']).flatten()
homeDogs = db[(db['Odds'] > 110) & (db['Location'] == 1)].dropna()[feature3Cols]
YhomeDogs = np.array(db[(db['Odds'] > 110) & (db['Location'] == 1)].dropna()['Result']).flatten()
roadDogs = db[(db['Odds'] > 110) & (db['Location'] == 0)].dropna()[feature3Cols]
YroadDogs = np.array(db[(db['Odds'] > 110) & (db['Location'] == 0)].dropna()['Result']).flatten()

x9_train, x9_test, y9_train, y9_test = train_test_split(X9, y, test_size=0.3, random_state=3)
x3_train, x3_test, y3_train, y3_test = train_test_split(X3, y, test_size=0.3, random_state=3)
dog_train, dog_test, Ydog_train, Ydog_test = train_test_split(dogs, Ydogs, test_size = 0.3, random_state = 3)
homeDog_train, homeDog_test, YhomeDog_train, YhomeDog_test = train_test_split(homeDogs, YhomeDogs, test_size = 0.3, random_state = 3)
roadDog_train, roadDog_test, YroadDog_train, YroadDog_test = train_test_split(roadDogs, YroadDogs, test_size = 0.3, random_state = 3)

In [218]:
clfgtb9 = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(x9_train, y9_train)
pred = clfgtb9.predict(x9_test)
print('All picks (R9):',clfgtb9.score(x9_test, y9_test))
print('All picks (R9) matrix:\n',confusion_matrix(y9_test, pred))
print(' ')
false_positive_rate, true_positive_rate, thresholds = roc_curve(y9_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)

clfgtb3 = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(x3_train, y3_train)
pred = clfgtb3.predict(x3_test)
print('All picks (R3):',clfgtb3.score(x3_test, y3_test))
print('All picks (R3) matrix:\n',confusion_matrix(y3_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(y3_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb3_dog = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(dog3_train, Ydog3_train)
pred = clfgtb3_dog.predict(dog3_test)
print('Dog (R3) picks:',clfgtb3_dog.score(dog3_test, Ydog3_test))
print('Dog (R3) picks matrix:\n',confusion_matrix(Ydog3_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ydog3_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb3_homedog = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(homeDog3_train, YhomeDog3_train)
pred = clfgtb3_homedog.predict(homeDog3_test)
print('Home Dog (R3) picks:',clfgtb3_homedog.score(homeDog3_test, YhomeDog3_test))
print('Home Dog (R3) picks matrix:\n',confusion_matrix(YhomeDog3_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(YhomeDog3_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb3_roaddog = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(roadDog3_train, YroadDog3_train)
pred = clfgtb3_roaddog.predict(roadDog3_test)
print('Road Dog (R3) picks:',clfgtb3_roaddog.score(roadDog3_test, YroadDog3_test))
print('Road Dog (R3) picks matrix:\n',confusion_matrix(YroadDog3_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(YroadDog3_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb9_dog = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(dog9_train, Ydog9_train)
pred = clfgtb9_dog.predict(dog9_test)
print('Dog (R9) picks:',clfgtb9_dog.score(dog9_test, Ydog9_test))
print('Dog (R9) picks matrix:\n',confusion_matrix(Ydog9_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ydog9_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb9_homedog = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(homeDog9_train, YhomeDog9_train)
pred = clfgtb9_homedog.predict(homeDog9_test)
print('Home (R9) Dog picks:',clfgtb9_homedog.score(homeDog9_test, YhomeDog9_test))
print('Home (R9) Dog picks matrix:\n',confusion_matrix(YhomeDog9_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(YhomeDog9_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb9_roaddog = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1, random_state=1).fit(roadDog9_train, YroadDog9_train)
pred = clfgtb9_roaddog.predict(roadDog9_test)
print('Road Dog (R9) picks:',clfgtb9_roaddog.score(roadDog9_test, YroadDog9_test))
print('Road Dog (R9) picks matrix:\n',confusion_matrix(YroadDog9_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(YroadDog9_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)

All picks (R9): 0.6210045662100456
All picks (R9) matrix:
 [[198 143]
 [106 210]]
 
0.6226010616578195
All picks (R3): 0.6164383561643836
All picks (R3) matrix:
 [[203 138]
 [114 202]]
0.6172742121088385
 
Dog (R3) picks: 0.6542635658914728
Dog (R3) picks matrix:
 [[404  25]
 [198  18]]
0.5125291375291375
 
Home Dog (R3) picks: 0.6590909090909091
Home Dog (R3) picks matrix:
 [[120  17]
 [ 58  25]]
0.5885586140181164
 
Road Dog (R3) picks: 0.6572769953051644
Road Dog (R3) picks matrix:
 [[271  18]
 [128   9]]
0.5017048468163564
 
Dog (R9) picks: 0.6775193798449612
Dog (R9) picks matrix:
 [[416  21]
 [187  21]]
0.5264533092765359
 
Home (R9) Dog picks: 0.6
Home (R9) Dog picks matrix:
 [[108  26]
 [ 62  24]]
0.542519958347796
 
Road Dog (R9) picks: 0.6643192488262911
Road Dog (R9) picks matrix:
 [[268  16]
 [127  15]]
0.5246478873239437


In [219]:
dogs2019 = db19[db19['Odds'] > 110].dropna().reset_index()
Ydogs2019 = db19[db19['Odds'] > 110].dropna()['Result']

In [220]:
print("Using dog data[rolling 9 averages]: \n")

print("Using 9 game standard predictor: ")
pred1 = clfgtb9.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb9.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred1))
print('')

print("Using 3 game standard predictor: ")
pred = clfgtb3.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb3.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R3 dog predictor: ")
pred = clfgtb3_dog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb3_dog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R3 home dog predictor: ")
pred = clfgtb3_homedog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb3_homedog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R3 road dog predictor: ")
pred = clfgtb3_roaddog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb3_roaddog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 dog predictor: ")
pred = clfgtb9_dog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb9_dog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 home dog predictor: ")
pred = clfgtb9_homedog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb9_homedog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 road dog predictor: ")
pred = clfgtb9_roaddog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb9_roaddog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

Using dog data[rolling 9 averages]: 

Using 9 game standard predictor: 
Dog picks: 0.6351648351648351
Dog picks matrix:
 [[484 150]
 [182  94]]

Using 3 game standard predictor: 
Dog picks: 0.6142857142857143
Dog picks matrix:
 [[441 193]
 [158 118]]

Using R3 dog predictor: 
Dog picks: 0.6912087912087912
Dog picks matrix:
 [[626   8]
 [273   3]]

Using R3 home dog predictor: 
Dog picks: 0.6802197802197802
Dog picks matrix:
 [[599  35]
 [256  20]]

Using R3 road dog predictor: 
Dog picks: 0.6879120879120879
Dog picks matrix:
 [[623  11]
 [273   3]]

Using R9 dog predictor: 
Dog picks: 0.6824175824175824
Dog picks matrix:
 [[607  27]
 [262  14]]

Using R9 home dog predictor: 
Dog picks: 0.6692307692307692
Dog picks matrix:
 [[585  49]
 [252  24]]

Using R9 road dog predictor: 
Dog picks: 0.6912087912087912
Dog picks matrix:
 [[619  15]
 [266  10]]



Very first model is the best one

In [221]:
odds = []
for i, n in enumerate(pred1):
    if n == 1:
        odds.append(dogs2019['Odds'][i])
        
print("Average odds:", statistics.mean(odds))
print("Median odds:", statistics.median(odds))


Average odds: 230.18032786885246
Median odds: 170.0


In [222]:
print("Using rolling 3 averages: \n")

print("Using 9 game standard predictor: ")
pred1 = clfgtb9.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb9.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred1))
print('')

print("Using 3 game standard predictor: ")
pred2 = clfgtb3.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb3.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred2))
print('')

print("Using R3 dog predictor: ")
pred = clfgtb3_dog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb3_dog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R3 home dog predictor: ")
pred = clfgtb3_homedog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb3_homedog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R3 road dog predictor: ")
pred = clfgtb3_roaddog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb3_roaddog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 dog predictor: ")
pred = clfgtb9_dog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb9_dog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 home dog predictor: ")
pred = clfgtb9_homedog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb9_homedog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 road dog predictor: ")
pred = clfgtb9_roaddog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb9_roaddog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

Using rolling 3 averages: 

Using 9 game standard predictor: 
Dog picks: 0.6197802197802198
Dog picks matrix:
 [[448 186]
 [160 116]]

Using 3 game standard predictor: 
Dog picks: 0.610989010989011
Dog picks matrix:
 [[429 205]
 [149 127]]

Using R3 dog predictor: 
Dog picks: 0.6692307692307692
Dog picks matrix:
 [[590  44]
 [257  19]]

Using R3 home dog predictor: 
Dog picks: 0.643956043956044
Dog picks matrix:
 [[535  99]
 [225  51]]

Using R3 road dog predictor: 
Dog picks: 0.667032967032967
Dog picks matrix:
 [[588  46]
 [257  19]]

Using R9 dog predictor: 
Dog picks: 0.6461538461538462
Dog picks matrix:
 [[522 112]
 [210  66]]

Using R9 home dog predictor: 
Dog picks: 0.6351648351648351
Dog picks matrix:
 [[507 127]
 [205  71]]

Using R9 road dog predictor: 
Dog picks: 0.6637362637362637
Dog picks matrix:
 [[566  68]
 [238  38]]



In [223]:
odds = []
for i, n in enumerate(pred1):
    if n == 1:
        odds.append(dogs2019['Odds'][i])
        
print("Average odds:", statistics.mean(odds))
print("Median odds:", statistics.median(odds))

Average odds: 252.72847682119206
Median odds: 177.5


Above: model 1 goes 116-186 with average betting odds of 252; this is profitable


In [419]:
db19 = pd.read_csv('2019stats.csv')
db = pd.read_csv('2018stats.csv')

In [420]:
db19.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)
db19['Date'] = db19['Date'].apply(str)
db19['Date'] = pd.to_datetime(db19.Date, format = "%Y%m%d")
db19 = db19.sort_values(by = ['Date']).reset_index(drop = True)

In [421]:
db['Date'] = db['Date'].apply(str)
db['Date'] = pd.to_datetime(db.Date, format= "%Y%m%d")
db = db.sort_values(by = ['Date']).reset_index(drop = True)
test = pd.concat([db, db19[:int(len(db19)/2)]], ignore_index = True)

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [422]:
# Leave in first half of prev season or cut it out?

test = test.drop(test.index[:int(len(db19)/2)]) 

In [423]:
db = test ### for simplicity

### Here is a realistic expectation of what a running database may look like. I added half the season's worth of 2019 onto the 2018 db, and dropped the first half of 2018. The model now looks at the second half of 2018 and the first half of 2019.

In [424]:
feature3Cols = ['Location','Rolling3EFG','Rolling3TOV','Rolling3ORB','Rolling3FTR','Rolling3Pace','Opp R3Pace','Opp R3EFG','Opp R3TOV','Opp R3ORB','Opp R3FTR']
feature9Cols = ['Location','Rolling9EFG','Rolling9TOV','Rolling9ORB','Rolling9FTR','Rolling9Pace','Opp R9Pace','Opp R9EFG','Opp R9TOV','Opp R9ORB','Opp R9FTR']

target = ['Result']
X3 = db.dropna()[feature3Cols]
X9 = db.dropna()[feature9Cols]
y = np.array(db.dropna()[target]).flatten()

dogs = db[db['Odds'] > 110].dropna()[feature3Cols]
Ydogs = np.array(db[db['Odds'] > 110].dropna()['Result']).flatten()
homeDogs = db[(db['Odds'] > 110) & (db['Location'] == 1)].dropna()[feature3Cols]
YhomeDogs = np.array(db[(db['Odds'] > 110) & (db['Location'] == 1)].dropna()['Result']).flatten()
roadDogs = db[(db['Odds'] > 110) & (db['Location'] == 0)].dropna()[feature3Cols]
YroadDogs = np.array(db[(db['Odds'] > 110) & (db['Location'] == 0)].dropna()['Result']).flatten()

x9_train, x9_test, y9_train, y9_test = train_test_split(X9, y, test_size=0.3, random_state=3)
x3_train, x3_test, y3_train, y3_test = train_test_split(X3, y, test_size=0.3, random_state=3)
dog_train, dog_test, Ydog_train, Ydog_test = train_test_split(dogs, Ydogs, test_size = 0.3, random_state = 3)
homeDog_train, homeDog_test, YhomeDog_train, YhomeDog_test = train_test_split(homeDogs, YhomeDogs, test_size = 0.3, random_state = 3)
roadDog_train, roadDog_test, YroadDog_train, YroadDog_test = train_test_split(roadDogs, YroadDogs, test_size = 0.3, random_state = 3)

In [425]:
clfgtb9 = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1, max_depth=1, random_state=3).fit(x9_train, y9_train)
pred = clfgtb9.predict(x9_test)
print('All picks (R9):',clfgtb9.score(x9_test, y9_test))
print('All picks (R9) matrix:\n',confusion_matrix(y9_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(y9_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb3 = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1, max_depth=1, random_state=1).fit(x3_train, y3_train)
pred = clfgtb3.predict(x3_test)
print('All picks (R3):',clfgtb3.score(x3_test, y3_test))
print('All picks (R3) matrix:\n',confusion_matrix(y3_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(y3_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb3_dog = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1, max_depth=1, random_state=1).fit(dog3_train, Ydog3_train)
pred = clfgtb3_dog.predict(dog3_test)
print('Dog (R3) picks:',clfgtb3_dog.score(dog3_test, Ydog3_test))
print('Dog (R3) picks matrix:\n',confusion_matrix(Ydog3_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ydog3_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb3_homedog = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1, max_depth=1, random_state=1).fit(homeDog3_train, YhomeDog3_train)
pred = clfgtb3_homedog.predict(homeDog3_test)
print('Home Dog (R3) picks:',clfgtb3_homedog.score(homeDog3_test, YhomeDog3_test))
print('Home Dog (R3) picks matrix:\n',confusion_matrix(YhomeDog3_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(YhomeDog3_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb3_roaddog = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1, max_depth=1, random_state=1).fit(roadDog3_train, YroadDog3_train)
pred = clfgtb3_roaddog.predict(roadDog3_test)
print('Road Dog (R3) picks:',clfgtb3_roaddog.score(roadDog3_test, YroadDog3_test))
print('Road Dog (R3) picks matrix:\n',confusion_matrix(YroadDog3_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(YroadDog3_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb9_dog = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1, max_depth=1, random_state=1).fit(dog9_train, Ydog9_train)
pred = clfgtb9_dog.predict(dog9_test)
print('Dog (R9) picks:',clfgtb9_dog.score(dog9_test, Ydog9_test))
print('Dog (R9) picks matrix:\n',confusion_matrix(Ydog9_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ydog9_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb9_homedog = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1, max_depth=1, random_state=1).fit(homeDog9_train, YhomeDog9_train)
pred = clfgtb9_homedog.predict(homeDog9_test)
print('Home (R9) Dog picks:',clfgtb9_homedog.score(homeDog9_test, YhomeDog9_test))
print('Home (R9) Dog picks matrix:\n',confusion_matrix(YhomeDog9_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(YhomeDog9_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
print(' ')

clfgtb9_roaddog = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1, max_depth=1, random_state=1).fit(roadDog9_train, YroadDog9_train)
pred = clfgtb9_roaddog.predict(roadDog9_test)
print('Road Dog (R9) picks:',clfgtb9_roaddog.score(roadDog9_test, YroadDog9_test))
print('Road Dog (R9) picks matrix:\n',confusion_matrix(YroadDog9_test, pred))
false_positive_rate, true_positive_rate, thresholds = roc_curve(YroadDog9_test, pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)

All picks (R9): 0.6331811263318112
All picks (R9) matrix:
 [[209 110]
 [131 207]]
0.633799224648031
 
All picks (R3): 0.6210045662100456
All picks (R3) matrix:
 [[210 109]
 [140 198]]
0.6220530132996976
 
Dog (R3) picks: 0.6511627906976745
Dog (R3) picks matrix:
 [[394  35]
 [190  26]]
0.5193926443926444
 
Home Dog (R3) picks: 0.6409090909090909
Home Dog (R3) picks matrix:
 [[113  24]
 [ 55  28]]
0.5810834579192683
 
Road Dog (R3) picks: 0.6525821596244131
Road Dog (R3) picks matrix:
 [[266  23]
 [125  12]]
0.5040032328947035
 
Dog (R9) picks: 0.6604651162790698
Dog (R9) picks matrix:
 [[401  36]
 [183  25]]
0.5189062224960395
 
Home (R9) Dog picks: 0.6136363636363636
Home (R9) Dog picks matrix:
 [[108  26]
 [ 59  27]]
0.5599618188129123
 
Road Dog (R9) picks: 0.6431924882629108
Road Dog (R9) picks matrix:
 [[258  26]
 [126  16]]
0.5105633802816901


In [426]:
dogs2019 = db19[int(len(db19)/2):][db19['Odds'] > 110].dropna().reset_index()
Ydogs2019 = db19[int(len(db19)/2):][db19['Odds'] > 110].dropna()['Result']

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


### This is now predicting all underdog games for last half of season

In [427]:
print("Using dog data[rolling 9 averages]: \n")

print("Using 9 game standard predictor: ")
pred1 = clfgtb9.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb9.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred1))
print('')

print("Using 3 game standard predictor: ")
pred2 = clfgtb3.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb3.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred2))
print('')

print("Using R3 dog predictor: ")
pred3 = clfgtb3_dog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb3_dog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred3))
print('')

print("Using R3 home dog predictor: ")
pred4 = clfgtb3_homedog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb3_homedog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred4))
print('')

print("Using R3 road dog predictor: ")
pred5 = clfgtb3_roaddog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb3_roaddog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred5))
print('')

print("Using R9 dog predictor: ")
pred6 = clfgtb9_dog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb9_dog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred6))
print('')

print("Using R9 home dog predictor: ")
pred7 = clfgtb9_homedog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb9_homedog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred7))
print('')

print("Using R9 road dog predictor: ")
pred8 = clfgtb9_roaddog.predict(dogs2019[feature9Cols])
print('Dog picks:',clfgtb9_roaddog.score(dogs2019[feature9Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred8))
print('')

Using dog data[rolling 9 averages]: 

Using 9 game standard predictor: 
Dog picks: 0.6440677966101694
Dog picks matrix:
 [[286  89]
 [100  56]]

Using 3 game standard predictor: 
Dog picks: 0.6440677966101694
Dog picks matrix:
 [[292  83]
 [106  50]]

Using R3 dog predictor: 
Dog picks: 0.6986817325800376
Dog picks matrix:
 [[369   6]
 [154   2]]

Using R3 home dog predictor: 
Dog picks: 0.6685499058380414
Dog picks matrix:
 [[338  37]
 [139  17]]

Using R3 road dog predictor: 
Dog picks: 0.6930320150659134
Dog picks matrix:
 [[366   9]
 [154   2]]

Using R9 dog predictor: 
Dog picks: 0.6911487758945386
Dog picks matrix:
 [[360  15]
 [149   7]]

Using R9 home dog predictor: 
Dog picks: 0.6516007532956686
Dog picks matrix:
 [[328  47]
 [138  18]]

Using R9 road dog predictor: 
Dog picks: 0.6949152542372882
Dog picks matrix:
 [[362  13]
 [149   7]]



In [428]:
print("Using rolling 3 averages: \n")

print("Using 9 game standard predictor: ")
pred1 = clfgtb9.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb9.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred1))
print('')

print("Using 3 game standard predictor: ")
pred2 = clfgtb3.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb3.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred2))
print('')

print("Using R3 dog predictor: ")
pred = clfgtb3_dog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb3_dog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R3 home dog predictor: ")
pred = clfgtb3_homedog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb3_homedog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R3 road dog predictor: ")
pred = clfgtb3_roaddog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb3_roaddog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 dog predictor: ")
pred = clfgtb9_dog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb9_dog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 home dog predictor: ")
pred = clfgtb9_homedog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb9_homedog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')

print("Using R9 road dog predictor: ")
pred = clfgtb9_roaddog.predict(dogs2019[feature3Cols])
print('Dog picks:',clfgtb9_roaddog.score(dogs2019[feature3Cols], Ydogs2019))
print('Dog picks matrix:\n',confusion_matrix(Ydogs2019, pred))
print('')
        

Using rolling 3 averages: 

Using 9 game standard predictor: 
Dog picks: 0.6214689265536724
Dog picks matrix:
 [[259 116]
 [ 85  71]]

Using 3 game standard predictor: 
Dog picks: 0.6252354048964218
Dog picks matrix:
 [[263 112]
 [ 87  69]]

Using R3 dog predictor: 
Dog picks: 0.6610169491525424
Dog picks matrix:
 [[342  33]
 [147   9]]

Using R3 home dog predictor: 
Dog picks: 0.6308851224105462
Dog picks matrix:
 [[303  72]
 [124  32]]

Using R3 road dog predictor: 
Dog picks: 0.672316384180791
Dog picks matrix:
 [[346  29]
 [145  11]]

Using R9 dog predictor: 
Dog picks: 0.6610169491525424
Dog picks matrix:
 [[320  55]
 [125  31]]

Using R9 home dog predictor: 
Dog picks: 0.615819209039548
Dog picks matrix:
 [[295  80]
 [124  32]]

Using R9 road dog predictor: 
Dog picks: 0.6534839924670434
Dog picks matrix:
 [[324  51]
 [133  23]]



In [429]:
odds = []
for i, n in enumerate(pred1):
    if n == 1:
        odds.append(dogs2019['Odds'][i])

print("Average odds:", statistics.mean(odds))
print("Median odds:", statistics.median(odds))

odds = []
for i, n in enumerate(pred2):
    if n == 1:
        odds.append(dogs2019['Odds'][i])

print("Average odds:", statistics.mean(odds))
print("Median odds:", statistics.median(odds))

Average odds: 282.75935828877004
Median odds: 200.0
Average odds: 264.78453038674036
Median odds: 190.0


In [418]:
pkl_filename = "GBC9model.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(clfgtb9, file)
    
pkl_filename = "GBC3model.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(clfgtb3, file)

This model drops 2018 data when adding first half of 2019.

The 9-game predictor goes 71-116 using 3-game average data, with average odds of 282.

The 3-game predictor goes 69-112 using 3-game average data, with average odds of 264.

The standard 9- or 3- game rolling models perform the best as opposed to the underdog training models. Training on dogs is making it 'scared' to pick underdogs when predicting new games bc most of the time they lose. I need some more revealing stats and some data analysis on when underdogs win.